För att kunna köra följande anteckningsböcker, om du inte redan har gjort det, behöver du ange openai-nyckeln i .env-filen som `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Nästa steg är att ladda in Embedding Index i en Pandas Dataframe. Embedding Index är lagrat i en JSON-fil som heter `embedding_index_3m.json`. Embedding Index innehåller embeddingar för varje YouTube-transkript fram till slutet av oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Nästa steg är att skapa en funktion som heter `get_videos` som kommer att söka i Embedding Index efter frågan. Funktionen kommer att returnera de 5 bästa videorna som är mest lik frågan. Funktionen fungerar på följande sätt:

1. Först skapas en kopia av Embedding Index.
2. Därefter beräknas Embedding för frågan med hjälp av OpenAI Embedding API.
3. Sedan skapas en ny kolumn i Embedding Index som heter `similarity`. Kolumnen `similarity` innehåller cosinuslikheten mellan frågans Embedding och Embedding för varje videosegment.
4. Nästa steg är att filtrera Embedding Index efter kolumnen `similarity`. Embedding Index filtreras för att endast inkludera videor som har en cosinuslikhet större än eller lika med 0,75.
5. Slutligen sorteras Embedding Index efter kolumnen `similarity` och de 5 bästa videorna returneras.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Den här funktionen är mycket enkel, den skriver bara ut resultaten av sökfrågan.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Först laddas Embedding Index in i en Pandas Dataframe.  
2. Därefter uppmanas användaren att skriva in en fråga.  
3. Sedan anropas funktionen `get_videos` för att söka i Embedding Index efter frågan.  
4. Slutligen anropas funktionen `display_results` för att visa resultaten för användaren.  
5. Användaren uppmanas sedan att skriva in en ny fråga. Denna process fortsätter tills användaren skriver `exit`.  

![](../../../../translated_images/sv/notebook-search.1e320b9c7fcbb0bc.webp)  

Du kommer att uppmanas att skriva in en fråga. Skriv in en fråga och tryck på enter. Applikationen kommer att returnera en lista med videor som är relevanta för frågan. Applikationen kommer också att returnera en länk till platsen i videon där svaret på frågan finns.  

Här är några frågor att prova:  

- Vad är Azure Machine Learning?  
- Hur fungerar konvolutionella neurala nätverk?  
- Vad är ett neuralt nätverk?  
- Kan jag använda Jupyter Notebooks med Azure Machine Learning?  
- Vad är ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfriskrivning**:
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen observera att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess modersmål bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för några missförstånd eller feltolkningar som uppstår till följd av användningen av denna översättning.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
